In [253]:
import pandas as pd
import numpy as np
import requests 
import html5lib
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from bs4 import BeautifulSoup 
import re 
from datetime import datetime, timedelta
import csv
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Firstly randomly subsample data on airline flight performance downloaded from the Bureau of Transportation Statistics. 



In [254]:
#set seed 
np.random.seed(666)

In [255]:
years = [2016, 2017, 2018]
months = [1, 3, 9,  12]

for year in years:
    for month in months:
        zipref = ZipFile("C:\\Users\\Ollie\\Downloads\\On_Time_Reporting_Carrier_On_Time_Performance_1987_present_" + f"{year}_"+f"{month}.zip","r") 
        zipref.extractall("C:\\Users\\Ollie\\Desktop\\BSE\\Courses\\Foundations of Data Science\\Project\\Data")
        print(f'{year}{month} Completed')

20161 Completed
20163 Completed
20169 Completed
201612 Completed
20171 Completed
20173 Completed
20179 Completed
201712 Completed
20181 Completed
20183 Completed
20189 Completed
201812 Completed


In [256]:
years = [2016, 2017, 2018]
months = [1, 3, 9,  12]

full_dfs=[]

for year in years:
    for month in months:
        df = pd.read_csv("C:\\Users\\Ollie\\Desktop\\BSE\\Courses\\Foundations of Data Science\\Project\\Data\\On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_" + f"{year}_"+f"{month}"+".csv", dtype=object) 
        full_dfs.append(df)
        print(f'{year}{month} Completed')

20161 Completed
20163 Completed
20169 Completed
201612 Completed
20171 Completed
20173 Completed
20179 Completed
201712 Completed
20181 Completed
20183 Completed
20189 Completed
201812 Completed


In [257]:
rs_dfs = []

for dfr in full_dfs:
    rs_df = dfr.sample(frac=0.04)
    rs_dfs.append(rs_df)
    

In [258]:
df  = pd.concat(rs_dfs)

In [259]:
keep = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'DOT_ID_Reporting_Airline', \
        'IATA_CODE_Reporting_Airline', 'OriginAirportID', 'Origin', 'OriginCityName', 'OriginStateName', \
       'OriginWac', 'DestAirportID', 'Dest', 'DestCityName', 'DestStateName', 'DestWac', 'CRSDepTime', \
       'DepDelay', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'CRSArrTime', 'ArrDelay', \
       'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Flights', \
        'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', \
        'LateAircraftDelay']

In [260]:
df_flght = df[keep]

In [26]:
df_flght.to_csv('C:\\Users\\Ollie\\Desktop\\BSE\\Courses\\Foundations of Data Science\\Project\\Data\\flight_data.csv', index=False)

In [5]:
df_flght= pd.read_csv('C:\\Users\\Ollie\\Desktop\\BSE\\Courses\\Foundations of Data Science\\Project\\Data\\flight_data.csv')

In [261]:
df_flght.shape


(234043, 38)

In [262]:
pd.to_datetime(df_flght['FlightDate'], format = '%Y-%m-%d')

347452   2016-01-07
8617     2016-01-29
372025   2016-01-17
227051   2016-01-23
321769   2016-01-08
            ...    
351185   2018-12-04
550170   2018-12-30
267448   2018-12-06
401401   2018-12-15
120876   2018-12-08
Name: FlightDate, Length: 234043, dtype: datetime64[ns]

In [263]:
df_flght.to_csv('C:\\Users\\Ollie\\Desktop\\BSE\\Courses\\Foundations of Data Science\\Project\\Data\\flight_data.csv', index=False)

### Download weather data from weather underground corresponding to the IATA airport code and flight date. 

In [61]:
# Do a for loop with station and date. Put in a dataframe and then column append the weather thing on IATA 
# flight date

In [ ]:
pd.to_datetime(df_flght['FlightDate'], format = '%Y-%m-%d')

In [20]:
airports = list(df_flght['Origin'])

In [33]:
gr_flght = df_flght.groupby(by='Origin')

In [233]:
### Scrape historical weather data from Weather Underground. 

def scrape_station(station, date):
    # Use .format(station, YYYY, M, D)
    URL = 'http://www.wunderground.com/history/daily/{}/date/{}-{}-{}.html'
    
    #Format date components.
    date_year = int(date[0:4])
    date_month = int(date[5:7])
    date_day = int(date[8:10])
    
    #format URl
    formatted_URL = URL.format(station, date_year, date_month, date_day)
    
    #Scrape weather data for specified station on specified date. 
    driver = webdriver.Chrome()
    driver.get(formatted_URL)
    tables = WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
    #Table 0 contains the data that we want. Hourly data is available in the second table.
    newTable = pd.read_html(tables[0].get_attribute('outerHTML'))
    return(newTable[0])
    #print(newTable[0].fillna(''))

In [234]:
airports = ['BOS', 'AUS']

In [225]:
dates = gr_flght.get_group('BOS')['FlightDate'].head()

In [226]:
dates

37     2016-01-10
81     2016-01-28
111    2016-01-08
122    2016-01-03
201    2016-01-14
Name: FlightDate, dtype: object

In [250]:
store_data = {}

#Scrape data for each airport, concatenate data for all dates for a given station, and put the dataframes
#for each station in a dictionary to be accessed later. 

#drop head once you've figured out how to put everything into the dictionary and merge => then run on 
#ASUS laptop over the course of a day (increase the wait time on the browser, also need to install 
#chrome, python, and anaconda on ASUS) then run!!!!
for airport in airports:
    df_airport_dates = []
    dates = gr_flght.get_group(airport)['FlightDate'].tail()
    airport_code = 'K' + airport
    # dates = list(gr_flght.get_group(airport)['FlightDate'])
    for date in dates:
        df_airport_date = scrape_station(airport_code, date)
        df_airport_date['FlightDate'] = np.tile(date, len(df_airport_date))
        df_airport_date.columns = ["Stats", "1", "2", "3", "4", "FlightDate"] 
        df_airport_date = df_airport_date.pivot(index=["FlightDate"], columns=["Stats"], values =["1"])
        df_airport_date['Origin'] = np.tile(airport, len(df_airport_date))
        print(df_airport_date)
        df_airport_dates.append(df_airport_date)
    df_airport_dates = pd.concat(df_airport_dates)
    df_airport_dates.reset_index(drop=True)
    store_data[airport] = df_airport_dates
    

                     1                                               \
Stats      Actual Time Astronomical Twilight Average Civil Twilight   
FlightDate                                                            
2018-12-26       9h 4m                   NaN   11.74            NaN   

                                                                   \
Stats      Day Average Temp Dew Point High High Temp Low Low Temp   
FlightDate                                                          
2018-12-26             32.7     11.74   23        40   3       26   

                                                                  \
Stats      Max Wind Speed Moon: waning gibbous Nautical Twilight   
FlightDate                                                         
2018-12-26             16                  NaN               NaN   

                                                                           \
Stats      Precipitation (past 24 hours from 05:54:00) Sea Level Pressure   
FlightDate 

                     1                                               \
Stats      Actual Time Astronomical Twilight Average Civil Twilight   
FlightDate                                                            
2018-12-17     10h 12m                   NaN    37.5            NaN   

                                                                   \
Stats      Day Average Temp Dew Point High High Temp Low Low Temp   
FlightDate                                                          
2018-12-17            53.25      37.5   42        72  32       38   

                                                                  \
Stats      Max Wind Speed Moon: waxing gibbous Nautical Twilight   
FlightDate                                                         
2018-12-17              7                  NaN               NaN   

                                                                           \
Stats      Precipitation (past 24 hours from 06:53:00) Sea Level Pressure   
FlightDate 

In [251]:
store_data['BOS']

1                                               \
Stats      Actual Time Astronomical Twilight Average Civil Twilight   
FlightDate                                                            
2018-12-26       9h 4m                   NaN   11.74            NaN   
2018-12-23         NaN                   NaN   17.83            NaN   
2018-12-21       9h 4m                   NaN   50.88            NaN   
2018-12-04      9h 15m                   NaN   15.88            NaN   
2018-12-26       9h 4m                   NaN   11.74            NaN   

                                                                      \
Stats      Day Average Temp Dew Point  High High Temp   Low Low Temp   
FlightDate                                                             
2018-12-26             32.7     11.74    23        40     3       26   
2018-12-23             36.5     17.83  21.0      41.0  13.0     34.0   
2018-12-21            53.37     50.88    61        64    40       44   
2018-12-04            35.13     15.88    29        42     8       28   
2018-12-26             32.7     11.74    23        40     3       26   

                                                                      \
Stats      Max Wind Speed Moon: waning crescent Moon: waning gibbous   
FlightDate                                                             
2018-12-26             16                   NaN                  NaN   
2018-12-23           17.0                   NaN                  NaN   
2018-12-21             28                   NaN                  NaN   
2018-12-04             18                   NaN                  NaN   
2018-12-26             16                   NaN                  NaN   

                                                   \
Stats      Moon: waxing gibbous Nautical Twilight   
FlightDate                                          
2018-12-26                  NaN               NaN   
2018-12-23                  NaN               NaN   
2018-12-21                  NaN               NaN   
2018-12-04                  NaN               NaN   
2018-12-26                  NaN               NaN   

                                                        \
Stats      Precipitation (past 24 hours from 05:54:00)   
FlightDate                                               
2018-12-26                                        0.00   
2018-12-23                                         NaN   
2018-12-21                                         NaN   
2018-12-04                                        0.00   
2018-12-26                                        0.00   

                                                                           \
Stats      Precipitation (past 24 hours from 11:54:00) Sea Level Pressure   
FlightDate                                                                  
2018-12-26                                         NaN              30.41   
2018-12-23                                        0.01              30.08   
2018-12-21                                        0.05              29.97   
2018-12-04                                         NaN              30.05   
2018-12-26                                         NaN              30.41   

                      Origin  
Stats      Visibility         
FlightDate                    
2018-12-26         10    BOS  
2018-12-23       10.0    BOS  
2018-12-21         10    BOS  
2018-12-04         10    BOS  
2018-12-26         10    BOS

In [252]:
store_data['AUS']

1                                               \
Stats      Actual Time Astronomical Twilight Average Civil Twilight   
FlightDate                                                            
2018-12-20     10h 11m                   NaN   35.88            NaN   
2018-12-08     10h 15m                   NaN   40.44            NaN   
2018-12-17     10h 12m                   NaN    37.5            NaN   
2018-12-11     10h 14m                   NaN   37.38            NaN   
2018-12-31     10h 13m                   NaN      43            NaN   

                                                                   \
Stats      Day Average Temp Dew Point High High Temp Low Low Temp   
FlightDate                                                          
2018-12-20            57.31     35.88   55        65  24       50   
2018-12-08            45.81     40.44   45        48  36       43   
2018-12-17            53.25      37.5   42        72  32       38   
2018-12-11            45.92     37.38   46        63  27       30   
2018-12-31            47.65        43   48        61  36       36   

                                                                       \
Stats      Max Wind Speed Moon: waning crescent Moon: waxing crescent   
FlightDate                                                              
2018-12-20             26                   NaN                   NaN   
2018-12-08             20                   NaN                   NaN   
2018-12-17              7                   NaN                   NaN   
2018-12-11             17                   NaN                   NaN   
2018-12-31             12                   NaN                   NaN   

                                                   \
Stats      Moon: waxing gibbous Nautical Twilight   
FlightDate                                          
2018-12-20                  NaN               NaN   
2018-12-08                  NaN               NaN   
2018-12-17                  NaN               NaN   
2018-12-11                  NaN               NaN   
2018-12-31                  NaN               NaN   

                                                        \
Stats      Precipitation (past 24 hours from 06:53:00)   
FlightDate                                               
2018-12-20                                        0.00   
2018-12-08                                         NaN   
2018-12-17                                        0.00   
2018-12-11                                        0.00   
2018-12-31                                         NaN   

                                                                           \
Stats      Precipitation (past 24 hours from 11:53:00) Sea Level Pressure   
FlightDate                                                                  
2018-12-20                                         NaN              29.49   
2018-12-08                                        2.84              29.69   
2018-12-17                                         NaN              29.69   
2018-12-11                                         NaN              29.81   
2018-12-31                                        0.29              29.47   

                      Origin  
Stats      Visibility         
FlightDate                    
2018-12-20         10    AUS  
2018-12-08         10    AUS  
2018-12-17         10    AUS  
2018-12-11         10    AUS  
2018-12-31         10    AUS

In [201]:
# Now figure out how to merge these into one dataset with column name Origin with airport number, then merge
# on Origin and Date, then save as a CSV. 


1                                               \
Stats      Actual Time Astronomical Twilight Average Civil Twilight   
Date                                                                  
2016-01-10         NaN                   NaN    62.7            NaN   
2016-01-28      10h 8m                   NaN    62.7            NaN   
2016-01-08      10h 8m                   NaN    62.7            NaN   
2016-01-03      10h 8m                   NaN    62.7            NaN   
2016-01-14      10h 8m                   NaN    62.7            NaN   

                                                                      \
Stats      Day Average Temp Dew Point  High High Temp   Low Low Temp   
Date                                                                   
2016-01-10            69.87      62.7  65.0      76.0  59.0     66.0   
2016-01-28            69.87      62.7    65        76    59       66   
2016-01-08            69.87      62.7    65        76    59       66   
2016-01-03            69.87      62.7    65        76    59       66   
2016-01-14            69.87      62.7    65        76    59       66   

                                                                  \
Stats      Max Wind Speed Moon: waxing gibbous Nautical Twilight   
Date                                                               
2016-01-10           24.0                  NaN               NaN   
2016-01-28             24                  NaN               NaN   
2016-01-08             24                  NaN               NaN   
2016-01-03             24                  NaN               NaN   
2016-01-14             24                  NaN               NaN   

                                                                           \
Stats      Precipitation (past 24 hours from 06:54:00) Sea Level Pressure   
Date                                                                        
2016-01-10                                         0.0              30.22   
2016-01-28                                        0.00              30.22   
2016-01-08                                        0.00              30.22   
2016-01-03                                        0.00              30.22   
2016-01-14                                        0.00              30.22   

                       
Stats      Visibility  
Date                   
2016-01-10       10.0  
2016-01-28         10  
2016-01-08         10  
2016-01-03         10  
2016-01-14         10

In [248]:
for airport in airports: 
    airports_weather = pd.concat(store_data.values(), ignore_index=False)

In [249]:
airports_weather.to_csv('C:\\Users\\Ollie\\Desktop\\BSE\\Courses\\Foundations of Data Science\\Project\\Data\\flight_data.csv', index=False)

1                                               \
Stats      Actual Time Astronomical Twilight Average Civil Twilight   
FlightDate                                                            
2016-01-10      9h 17m                   NaN    48.5            NaN   
2016-01-28      9h 49m                   NaN   21.46            NaN   
2016-01-08         NaN                   NaN    29.3            NaN   
2016-01-03       9h 9m                   NaN   22.21            NaN   
2016-01-14      9h 23m                   NaN   11.13            NaN   
2016-01-18     10h 27m                   NaN   34.54            NaN   
2016-01-16     10h 25m                   NaN      37            NaN   
2016-01-09     10h 19m                   NaN   34.15            NaN   
2016-01-21     10h 30m                   NaN   46.33            NaN   
2016-01-22     10h 32m                   NaN   29.13            NaN   

                                                                      ...  \
Stats      Day Average Temp Dew Point  High High Temp   Low Low Temp  ...   
FlightDate                                                            ...   
2016-01-10            49.48      48.5    57        58    37       39  ...   
2016-01-28            33.58     21.46    29        42    15       27  ...   
2016-01-08            36.07      29.3  34.0      42.0  24.0     30.0  ...   
2016-01-03            36.58     22.21    24        43    20       31  ...   
2016-01-14            25.43     11.13    16        29     5       21  ...   
2016-01-18            42.71     34.54    39        57    28       30  ...   
2016-01-16            47.27        37    44        64    32       37  ...   
2016-01-09            49.04     34.15    41        59    29       39  ...   
2016-01-21            54.21     46.33    56        65    28       44  ...   
2016-01-22            43.38     29.13    30        54    24       28  ...   

                                                   \
Stats      Moon: waxing gibbous Nautical Twilight   
FlightDate                                          
2016-01-10                  NaN               NaN   
2016-01-28                  NaN               NaN   
2016-01-08                  NaN               NaN   
2016-01-03                  NaN               NaN   
2016-01-14                  NaN               NaN   
2016-01-18                  NaN               NaN   
2016-01-16                  NaN               NaN   
2016-01-09                  NaN               NaN   
2016-01-21                  NaN               NaN   
2016-01-22                  NaN               NaN   

                                                        \
Stats      Precipitation (past 24 hours from 05:54:00)   
FlightDate                                               
2016-01-10                                         NaN   
2016-01-28                                        0.00   
2016-01-08                                         0.0   
2016-01-03                                        0.00   
2016-01-14                                        0.00   
2016-01-18                                         NaN   
2016-01-16                                         NaN   
2016-01-09                                         NaN   
2016-01-21                                         NaN   
2016-01-22                                         NaN   

                                                        \
Stats      Precipitation (past 24 hours from 06:05:00)   
FlightDate                                               
2016-01-10                                         NaN   
2016-01-28                                         NaN   
2016-01-08                                         NaN   
2016-01-03                                         NaN   
2016-01-14                                         NaN   
2016-01-18                                         NaN   
2016-01-16                                         NaN   
2016-01-09                                         NaN   
2016-01-21  

In [ ]:
#if airports don't return on Weather Undeground make a list of airport names that don't return.
# get airport names from here: https://github.com/datasets/airport-codes/tree/master/data. 

#They will return, I just need to add a K to the start of each string to identify it as an airport code.

In [ ]:
#want the daily weather for each airport on that date.Can use the summary data on each page. 
#Use beautiful soup to get sea level pressure, max wind speed, visibility, precipitation, and high and 
#low temp.

In [74]:
airports_weather = []
for airport in airports:
    airports_weather.append('K' + airport)

In [ ]:
for date in dates:
    date_year = int(date_min[0:4])
    date_month = int(date_min[5:7])
    date_day = int(date_min[8:10])

In [ ]:
#Want to automate this process over every airport and for every relevant date. 

In [27]:
url = 'https://www.wunderground.com/history/daily/KBOS/date/2022-11-6'
driver = webdriver.Chrome()
driver.get(url)
tables = WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
#Table 0 contains the data that we want. Hourly data is available in the second table.
newTable = pd.read_html(tables[0].get_attribute('outerHTML'))
print(newTable[0].fillna(''))

                               Temperature (°F)     Actual Historic Avg.  \
                             Precipitation (in)     Actual Historic Avg.   
                                 Dew Point (°F)     Actual Historic Avg.   
                                     Wind (mph)     Actual Historic Avg.   
                        Sea Level Pressure (in)     Actual Historic Avg.   
                                      Astronomy Day Length          Rise   
0                                     High Temp         76          54.9   
1                                      Low Temp         66          40.8   
2                              Day Average Temp      69.87          47.8   
3   Precipitation (past 24 hours from 06:54:00)       0.00          4.60   
4                                     Dew Point       62.7             -   
5                                          High         65             -   
6                                           Low         59             -   
7           

In [98]:
newTable[0]

,Temperature (°F),Actual,Historic Avg.,Record,Polygon
,Precipitation (in),Actual,Historic Avg.,Record,Polygon
,Dew Point (°F),Actual,Historic Avg.,Record,Polygon
,Wind (mph),Actual,Historic Avg.,Record,Polygon
,Sea Level Pressure (in),Actual,Historic Avg.,Record,Polygon
,Astronomy,Day Length,Rise,Set,Polygon
0,High Temp,76,54.9,73,NaN
1,Low Temp,66,40.8,21,NaN
2,Day Average Temp,69.87,47.8,-,NaN
3,Precipitation (past 24 hours from 06:54:00),0.00,4.60,-,NaN
4,Dew Point,62.7,-,-,NaN


In [65]:
URL = 'https://www.wunderground.com/history/daily/KBOS/date/2022-11-6'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())

html = urlopen(URL).read().decode('utf-8')


<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/Organization" lang="en" prefix="og: http://ogp.me/ns#">
 <head itemscope="" itemtype="http://schema.org/WebSite">
  <meta charset="utf-8"/>
  <title>
   East Boston, MA Weather History | Weather Underground
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="//widgets.outbrain.com" rel="dns-prefetch"/>
  <link href="//odb.outbrain.com" rel="dns-prefetch"/>
  <link href="//c.amazon-adsystem.com" rel="dns-prefetch"/>
  <link href="//s.amazon-adsystem.com" rel="dns-prefetch"/>
  <link href="//aax.amazon-adsystem.com" rel="dns-prefetch"/>
  <link href="//partner.googleadservices.com" rel="dns-prefetch"/>
  <link href="//tpc.googlesyndication.com" rel="dns-prefetch"/>
  <link href="//pagead2.googlesyndication.com" rel="dns-prefetch"/>
  <link href="//h.nexac.com" rel="dns-prefetch"/>
  <link href="//js-sec.indexww.com" 

In [67]:
print(html)

AttributeError: 'str' object has no attribute 'prettify'

In [68]:
soup = BeautifulSoup(html, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/Organization" lang="en" prefix="og: http://ogp.me/ns#">
 <head itemscope="" itemtype="http://schema.org/WebSite">
  <meta charset="utf-8"/>
  <title>
   East Boston, MA Weather History | Weather Underground
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="//widgets.outbrain.com" rel="dns-prefetch"/>
  <link href="//odb.outbrain.com" rel="dns-prefetch"/>
  <link href="//c.amazon-adsystem.com" rel="dns-prefetch"/>
  <link href="//s.amazon-adsystem.com" rel="dns-prefetch"/>
  <link href="//aax.amazon-adsystem.com" rel="dns-prefetch"/>
  <link href="//partner.googleadservices.com" rel="dns-prefetch"/>
  <link href="//tpc.googlesyndication.com" rel="dns-prefetch"/>
  <link href="//pagead2.googlesyndication.com" rel="dns-prefetch"/>
  <link href="//h.nexac.com" rel="dns-prefetch"/>
  <link href="//js-sec.indexww.com" 

In [69]:
content = soup.find(id="inner-content")

In [70]:
 content_main = content.find(class_="region-content-main")

In [71]:
print(content_main.prettify())

<div _ngcontent-sc285="" class="region-content-main">
 <div _ngcontent-sc285="" class="row">
  <div _ngcontent-sc285="" class="small-12 columns has-sidebar">
   <div _ngcontent-sc285="" class="row">
    <div _ngcontent-sc285="" class="columns small-12">
     <lib-link-selector _ngcontent-sc285="" _nghost-sc225="">
      <div _ngcontent-sc225="" class="selector-container">
       <div _ngcontent-sc225="" class="selector-border">
        <div _ngcontent-sc225="" class="selector">
         <!-- -->
         <a _ngcontent-sc225="" class="link selected ng-star-inserted" href="/history/daily/KBOS/date/2022-11-6">
          Daily
         </a>
         <!-- -->
         <!-- -->
         <!-- -->
         <a _ngcontent-sc225="" class="link ng-star-inserted" href="/history/weekly/KBOS/date/2022-11-6">
          Weekly
         </a>
         <!-- -->
         <!-- -->
         <!-- -->
         <a _ngcontent-sc225="" class="link ng-star-inserted" href="/history/monthly/KBOS/date/2022-11">
     

In [57]:
print(content.prettify())

<section _ngcontent-sc165="" class="inner-content" id="inner-content">
 <div _ngcontent-sc286="" class="region-content-top">
  <ad-wx-ws _ngcontent-sc286="" class="ng-star-inserted">
   <div class="top-banner ad-wrap ad-desktop" id="top-banner-ad-browser">
    <div id="WX_WindowShade">
    </div>
   </div>
  </ad-wx-ws>
  <!-- -->
  <ad-mw-position-1 _ngcontent-sc286="" _nghost-sc116="" class="ng-star-inserted">
   <div _ngcontent-sc116="" class="top-banner ad-wrap ad-mobile" id="position-1-banner-ad-browser">
    <div _ngcontent-sc116="" id="MW_Position1">
    </div>
   </div>
  </ad-mw-position-1>
  <!-- -->
  <lib-city-header _ngcontent-sc286="" _nghost-sc238="">
   <div _ngcontent-sc238="" class="row">
    <div _ngcontent-sc238="" class="columns small-12 city-header ng-star-inserted">
     <span _ngcontent-sc238="" class="subheading">
      <!-- -->
      <strong _ngcontent-sc238="">
       42.38
      </strong>
      °N,
      <strong _ngcontent-sc238="">
       71.04
      </stro

In [53]:
soup.find_all("div", class_="summary-title")

[<div _ngcontent-sc257="" class="summary-title">Summary</div>]

In [47]:
print(html)

[<div _ngcontent-sc257="" class="summary-table"> No data recorded <!-- --><!-- --></div>]


In [ ]:
trs = soup.find("div", attrs={"class": "clear data-table sortable-header dashed-table-tr alternate-rows"}).find("table").findAll("tr")
for tr in trs:
    tds = tr.findAll("td")
    if len(tds) < 1:
        continue
    name = tds[0].text
    print name
f.close()

In [ ]:
### Scrape historical weather data from Weather Underground. 

def scrape_station(station):
    start_date = datetime(year=min_year, month=min_month, day=min_day)
    end_date = datetime(year=max_year, month=max_month, day=max_day)

    # Make sure a directory exists for the station web pages
    os.mkdir(station)

    # Use .format(station, YYYY, M, D)
    URL = 'http://www.wunderground.com/history/daily/{}/date/{}-{}-{}.html'

    while start_date != end_date:

        if start_date.day == 1:
            print(start_date)

        formatted_URL = URL.format(station, start_date.year, start_date.month, start_date.day)
        html = urlopen(formatted_URL).read().decode('utf-8')

        out_file_name = '{}/{}-{}-{}.html'.format(station, start_date.year, start_date.month,start_date.day)

        with open(out_file_name, 'w') as out_file:
            out_file.write(html)

        current_date += timedelta(days=1)


In [ ]:
# Scrape the stations that appear in the flights dataset
for station in ['KCLT', 'KCQT', 'KHOU', 'KIND', 'KJAX',
                'KMDW', 'KNYC', 'KPHL', 'KPHX', 'KSEA']:
    scrape_station(station)